# London Crime Data

### Table of Contents

* [1 - Importing the data](#1)
    * [1.1 - Importing the data](#1.1)
    * [1.2 - Exploring the dataset](#1.2)
* [2 - Preprocessing/Data cleaning](#2)
    * [2.1 - Feature selection](#2.1)
    * [2.2 - Handling missing/null values](#2.2)
* [3 - Model selection](#3)
    * [3.1 - Logistic regression](#3.1)
    * [3.2 - Support vector machine - SVM](#3.2)
    * [3.2 - Random forest classifier](#3.2)
* [4 - Results](#4)    

Collecting the data via API

In [23]:
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects


url_cc = 'https://data.police.uk/api/crime-categories'
url_cl = 'https://data.police.uk/api/crimes-at-location'
crime_cat = list()

parameters = {'lat':51.487884,'lng':-0.205808}
keys = ['category', 'month', 'location']

try:
    # get crime_categories
    response = requests.get(url_cc)
    response_2 = requests.get(url_cl, params=parameters) 
    # data = json.loads(response.text)
    for i in range(len(response.json())):
        crime_cat.append(response.json()[i]['name'])
        # print(response.json()[i]['name'])
    print(crime_cat)
    print(f"len response_2 = {len(response_2.json())}")
    for key in keys:
        print(f"{key} : {response_2.json()[0][key]}")
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)


['All crime', 'Anti-social behaviour', 'Bicycle theft', 'Burglary', 'Criminal damage and arson', 'Drugs', 'Other theft', 'Possession of weapons', 'Public order', 'Robbery', 'Shoplifting', 'Theft from the person', 'Vehicle crime', 'Violence and sexual offences', 'Other crime']
len response_2 = 1
category : vehicle-crime
month : 2020-12
location : {'latitude': '51.487671', 'street': {'id': 950134, 'name': 'On or near Sun Road'}, 'longitude': '-0.204974'}


Putting together all the CSV files with the crime data

In [15]:
import os
import pandas as pd
data = pd.DataFrame()
rootdir = 'C:/Users/CarlosFelipe/Dropbox/DataScience/DataScienceProjects/Unguided/LondonCrimeAnalysis/input'
"""
In each folder we have 3 types of files:
1-aaaa-mm-metropolitan-outcomes.csv
2-aaaa-mm-metropolitan-stop-and-search.csv
3-aaaa-mm-metropolitan-street.csv
"""
type_1 = 'outcomes'
type_2 = 'stop-and-search'
type_3 = 'street'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if type_1 in file:
            #print(type_1 + ' : ' + file)
            aux = pd.read_csv(os.path.join(subdir, file))
            data = data.append(aux)
        elif type_2 in file:
            #print(type_2 + ' : ' + file)
            pass
        elif type_3 in file:
            #print(type_3 + ' : ' + file)
            pass
        else:
            print("NEW TYPE OF FILE")
data.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
0,ed18f67c604a62bbce802a4da718a252127dc60b24ae9c...,2019-05,Metropolitan Police Service,Metropolitan Police Service,NaN,NaN,No location,NaN,NaN,Suspect charged
1,bfcc3013a712b367287fdac468d460db4899b9aba45bef...,2019-05,Metropolitan Police Service,Metropolitan Police Service,NaN,NaN,No location,NaN,NaN,Suspect charged
2,3a3890b980964f6c4cfb4e32b9ff7914b71521597889ae...,2019-05,Metropolitan Police Service,Metropolitan Police Service,-0.033488,51.531997,On or near DANE PLACE,E01004228,Tower Hamlets 003A,Suspect charged
3,2f59143e13f9bd3c5e56a36eb772fa95ea23f3a2377e9e...,2019-05,Metropolitan Police Service,Metropolitan Police Service,NaN,NaN,No location,NaN,NaN,Suspect charged
4,c5295ea1b6e4a7ea4470f7d7560546b749ba1393dc79b8...,2019-05,Metropolitan Police Service,Metropolitan Police Service,NaN,NaN,No location,NaN,NaN,Suspect charged
